### Model Training for binary classification of aliens vs. predators

The CNN model, that will be trained, receives images in shape of (64,63,3) as input. By applying the Keras library, I was able to use the ImageDataGenerator class and create objects of it for training and validation data, because the folder structure of these given data is suitable for the ImageDataGenerator usage.

In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

Using TensorFlow backend.


In [2]:
maxsize = (64,64)

The ImageDataGenerator object for training data includes pixel value standardization. It generate batches of tensor image data with real-time data augmentation, inter alia by the horizontal flip. Shearing can be powerful to deal with different orientations, thus a maximum range of shearing is defined. A maximum zoom range is declared too, which completes the applied techniques of data augmentation for the training set. For the validation data, pixel value standardization has to be applied too, because the pixel values must be within the same range compared to the training data, which the model will be used too.

There are overall 694 images belonging to both classes (aliens and predators), data augmentation will increase the amount of data for training set for the model training. 200 images are found in the validation set.

In [3]:
train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.1,
                horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
training_set = train_datagen.flow_from_directory(
                            'dataset/train', 
                            target_size=maxsize,
                            batch_size=128, 
                            class_mode='binary')

Found 694 images belonging to 2 classes.


In [5]:
validation_set = validation_datagen.flow_from_directory(
                                    'dataset/validation',
                                    target_size=maxsize,
                                    batch_size=128,
                                    class_mode='binary')

Found 200 images belonging to 2 classes.


A Sequential object is created. The model architecture is displayed below. During the hyperparameter tuning and experiments of different architectures, I figured out that a shallow network with 1 Conv2D layer often reaches its limit of accuracy after 15, 20 epochs. A deeper network with multiple Conv2D layers manages to reach a training accuracy of almost 98%, but overfitting occurs very often. That is the reason why I added multiple dropout layers after each Max-Pooling layer and Flatten layer, especially with a relatively huge dropout rate (0.4). 

Another aspect that I would like to mention: When I added the Dropout layer between the Conv2D and Max-Pooling layer, the model performance will be heavily affected, as the accuracy will stick around 50%. 

In [6]:
model = Sequential()
model.add(Conv2D(filters=32, 
                 kernel_size=(3,3), 
                 input_shape=(64,64,3),
                 activation='relu',
                 data_format='channels_last'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
dropout (Dropout)            (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['acc'])

I added an early-stopping callback, which is based on the validation accuracy. I noticed that the validation accuracy often reaches 79%-82% and then stops to increase. So I want to stop the training once a validation accuracy of 86% is reached, which is the highest value that I have observed during many model trainings.

In [9]:
class earlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc')>0.86):
            print("\nSUCCESS: Reached 86% validation accuracy, I will stop the training earlier!")
            self.model.stop_training = True
earlyStopping = earlyStoppingCallback()

In [10]:
model.fit_generator(
        training_set,
        steps_per_epoch=100,
        epochs=20,
        callbacks = [earlyStopping],
        validation_data=validation_set,
        validation_steps=100)

Epoch 1/20
100/100 [==============================] - 92s 924ms/step - loss: 0.6404 - acc: 0.6018 - val_loss: 0.6174 - val_acc: 0.5750
Epoch 2/20
100/100 [==============================] - 86s 859ms/step - loss: 0.5300 - acc: 0.7416 - val_loss: 0.5639 - val_acc: 0.7050
Epoch 3/20
100/100 [==============================] - 89s 889ms/step - loss: 0.4601 - acc: 0.7824 - val_loss: 0.5043 - val_acc: 0.7450
Epoch 4/20
100/100 [==============================] - 91s 905ms/step - loss: 0.4044 - acc: 0.8158 - val_loss: 0.4700 - val_acc: 0.7750
Epoch 5/20
100/100 [==============================] - 95s 946ms/step - loss: 0.3668 - acc: 0.8360 - val_loss: 0.4512 - val_acc: 0.8050
Epoch 6/20
100/100 [==============================] - 98s 982ms/step - loss: 0.3408 - acc: 0.8487 - val_loss: 0.4835 - val_acc: 0.7650
Epoch 7/20
100/100 [==============================] - 93s 928ms/step - loss: 0.3064 - acc: 0.8623 - val_loss: 0.4902 - val_acc: 0.7600
Epoch 8/20
100/100 [==============================] - 1

In [11]:
model.save('model-91-86.h5')